# मॉडल प्रूनिंग मशाल अभ्यास
पाइटोरच ने संस्करण 1.4.0 के बाद से प्रूनिंग ऑपरेशंस को जोड़ा है। `torch.nn.utils.prune` मॉड्यूल में, इस ट्यूटोरियल को प्रूनिंग स्कोप के अनुसार विभाजित किया गया है और निम्नलिखित प्रूनिंग विधियों में विभाजित किया गया है:
- स्थानीय छंटाई
  - संरचित छंटाई
    - यादृच्छिक संरचित छंटाई (यादृच्छिक_संरचित)
    - सामान्य संरचित छंटाई (ln_structured)
  - असंरचित छंटाई
    - यादृच्छिक असंरचित छंटाई (यादृच्छिक_असंरचित)
    - सामान्य असंरचित छंटाई (l1_unstructured)
- ग्लोबल प्रूनिंग
  - असंरचित छंटाई (वैश्विक_असंरचित)
- कस्टम प्रूनिंग
  
**ध्यान दें:** ग्लोबल प्रूनिंग केवल असंरचित प्रूनिंग का समर्थन करती है।

## 1. आंशिक छंटाई
सबसे पहले, हम स्थानीय प्रूनिंग विधि का परिचय देते हैं, जो नेटवर्क की एकल परत या स्थानीय श्रेणी की प्रूनिंग को संदर्भित करता है।

### 1.1 संरचित छंटाई
प्रूनिंग विधि के अनुसार, इसे संरचित प्रूनिंग और असंरचित प्रूनिंग में विभाजित किया जा सकता है। असंरचित छंटाई बेतरतीब ढंग से कुछ वजन मापदंडों को 0 में बदल देगी, जबकि संरचित छंटाई एक निश्चित आयाम में कुछ चैनलों को 0 में बदल देगी।

#### 1.1.1 यादृच्छिक संरचित छंटाई (यादृच्छिक_संरचित)

In [343]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
from torchsummary import summary

एक क्लासिक LeNet नेटवर्क बनाएं

In [344]:
# LeNet नेटवर्क को परिभाषित करें
class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=16 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=num_classes)

    def forward(self, x):
        x = self.maxpool(F.relu(self.conv1(x)))
        x = self.maxpool(F.relu(self.conv2(x)))

        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet().to(device=device)

In [345]:
#प्रिंट मॉडल संरचना
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 24, 24]             156
         MaxPool2d-2            [-1, 6, 12, 12]               0
            Conv2d-3             [-1, 16, 8, 8]           2,416
         MaxPool2d-4             [-1, 16, 4, 4]               0
            Linear-5                  [-1, 120]          30,840
            Linear-6                   [-1, 84]          10,164
            Linear-7                   [-1, 10]             850
Total params: 44,426
Trainable params: 44,426
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 0.17
Estimated Total Size (MB): 0.22
----------------------------------------------------------------


In [346]:
#पहली संकेंद्रित परत के मापदंडों को प्रिंट करें
module = model.conv1
print(list(module.named_parameters()))

[('weight', Parameter containing:
tensor([[[[ 0.0220,  0.1789, -0.0544, -0.0713,  0.0478],
          [ 0.1995, -0.0415,  0.0288, -0.1431,  0.1057],
          [ 0.1600,  0.0248, -0.1903, -0.0242, -0.1961],
          [-0.0211,  0.0257, -0.1116, -0.1678,  0.0611],
          [ 0.0012,  0.0420, -0.1725, -0.1265, -0.1075]]],


        [[[-0.0540, -0.1928, -0.0355, -0.0075, -0.1481],
          [ 0.0135,  0.0192,  0.0082, -0.0120, -0.0164],
          [-0.0435, -0.1488,  0.1092, -0.0041,  0.1960],
          [-0.1045, -0.0136,  0.0398, -0.1286,  0.0617],
          [-0.0091,  0.0466,  0.1827,  0.1655,  0.0727]]],


        [[[ 0.1216, -0.0833, -0.1491, -0.1143,  0.0113],
          [ 0.0452,  0.1662, -0.0425, -0.0904, -0.1235],
          [ 0.0565,  0.0933, -0.0721,  0.0909,  0.1837],
          [-0.1739,  0.0263,  0.1339,  0.0648, -0.0382],
          [-0.1667,  0.1478,  0.0448, -0.0892,  0.0815]]],


        [[[ 0.1976,  0.0123,  0.1523, -0.1207,  0.1493],
          [-0.1799,  0.0580,  0.1490,  0.1

In [347]:
#मॉड्यूल में विशेषता टेंसर नामित_बफ़र्स प्रिंट करें, जो प्रारंभ में एक खाली सूची है।
print(list(module.named_buffers()))

[]


In [348]:
#मॉडल के राज्य शब्दकोश को प्रिंट करें राज्य शब्दकोश में सभी पैरामीटर शामिल हैं।
print(model.state_dict().keys())

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])


In [349]:
# पहला पैरामीटर: मॉड्यूल, कांट-छांट किए जाने वाले विशिष्ट मॉड्यूल का प्रतिनिधित्व करता है, यहां मॉड्यूल=model.conv1 को संदर्भित करता है,
# समझाएं कि प्रूनिंग पहली कन्वोल्यूशनल परत पर की जानी है।
# दूसरा पैरामीटर: नाम, दर्शाता है कि चयनित मॉड्यूल में कौन से पैरामीटर को काटा जाना है।
# यह नाम = "वजन" पर सेट है, जिसका अर्थ है कि नेटवर्क में वजन कम किया गया है, पूर्वाग्रह नहीं।
# तीसरा पैरामीटर: राशि, मॉडल में एक विशिष्ट अनुपात या मापदंडों की पूर्ण संख्या पर की जाने वाली छंटाई का प्रतिनिधित्व करता है।
# राशि 0.0-1.0 के बीच एक फ्लोट मान है, जो अनुपात का प्रतिनिधित्व करता है, या एक सकारात्मक पूर्णांक है, जो ट्रिम किए जाने वाले मापदंडों की निर्दिष्ट संख्या का प्रतिनिधित्व करता है।
# चौथा पैरामीटर: मंद, कांट-छांट किए जाने वाले चैनल के आयाम सूचकांक का प्रतिनिधित्व करता है।
#            

prune.random_structured(module, name="weight", amount=2, dim=0)

Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))

In [350]:
# मॉडल के राज्य शब्दकोश को फिर से प्रिंट करें और conv1 परत का निरीक्षण करें
print(model.state_dict().keys())

odict_keys(['conv1.bias', 'conv1.weight_orig', 'conv1.weight_mask', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])


In [351]:
#मॉड्यूल में विशेषता टेंसर नामांकित_बफ़र्स को फिर से प्रिंट करें
print(list(module.named_parameters()))

[('bias', Parameter containing:
tensor([-0.0893, -0.1464, -0.1101, -0.0076,  0.1493, -0.0418],
       requires_grad=True)), ('weight_orig', Parameter containing:
tensor([[[[ 0.0220,  0.1789, -0.0544, -0.0713,  0.0478],
          [ 0.1995, -0.0415,  0.0288, -0.1431,  0.1057],
          [ 0.1600,  0.0248, -0.1903, -0.0242, -0.1961],
          [-0.0211,  0.0257, -0.1116, -0.1678,  0.0611],
          [ 0.0012,  0.0420, -0.1725, -0.1265, -0.1075]]],


        [[[-0.0540, -0.1928, -0.0355, -0.0075, -0.1481],
          [ 0.0135,  0.0192,  0.0082, -0.0120, -0.0164],
          [-0.0435, -0.1488,  0.1092, -0.0041,  0.1960],
          [-0.1045, -0.0136,  0.0398, -0.1286,  0.0617],
          [-0.0091,  0.0466,  0.1827,  0.1655,  0.0727]]],


        [[[ 0.1216, -0.0833, -0.1491, -0.1143,  0.0113],
          [ 0.0452,  0.1662, -0.0425, -0.0904, -0.1235],
          [ 0.0565,  0.0933, -0.0721,  0.0909,  0.1837],
          [-0.1739,  0.0263,  0.1339,  0.0648, -0.0382],
          [-0.1667,  0.1478,  0.

In [352]:
#मॉड्यूल में विशेषता टेंसर नामांकित_बफ़र्स को फिर से प्रिंट करें
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]],


        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]],


        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]]]))

निष्कर्ष: प्रूनिंग ऑपरेशन के बाद, मूल वेट मैट्रिक्स वेट वेट_ओरिग बन जाता है। और मॉड्यूल.नाम_बफ़र्स (), जिसने प्रूनिंग से पहले एक खाली सूची मुद्रित की थी, अब एक वेट_मास्क पैरामीटर है।

In [353]:
#मॉड्यूल.वेट प्रिंट करें और देखें कि आपको क्या मिला?
print(module.weight)

tensor([[[[ 0.0000,  0.0000, -0.0000, -0.0000,  0.0000],
          [ 0.0000, -0.0000,  0.0000, -0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000, -0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0000, -0.0000, -0.0000]]],


        [[[-0.0540, -0.1928, -0.0355, -0.0075, -0.1481],
          [ 0.0135,  0.0192,  0.0082, -0.0120, -0.0164],
          [-0.0435, -0.1488,  0.1092, -0.0041,  0.1960],
          [-0.1045, -0.0136,  0.0398, -0.1286,  0.0617],
          [-0.0091,  0.0466,  0.1827,  0.1655,  0.0727]]],


        [[[ 0.1216, -0.0833, -0.1491, -0.1143,  0.0113],
          [ 0.0452,  0.1662, -0.0425, -0.0904, -0.1235],
          [ 0.0565,  0.0933, -0.0721,  0.0909,  0.1837],
          [-0.1739,  0.0263,  0.1339,  0.0648, -0.0382],
          [-0.1667,  0.1478,  0.0448, -0.0892,  0.0815]]],


        [[[ 0.0000,  0.0000,  0.0000, -0.0000,  0.0000],
          [-0.0000,  0.0000,  0.0000,  0.0000, -0.0000],
          [-0.0000,

निष्कर्ष: प्रूनिंग ऑपरेशन के बाद, मूल वजन वेट_ओरिग बन जाता है और नामित_पैरामीटर में संग्रहीत होता है। संबंधित प्रूनिंग मैट्रिक्स को वेट_मास्क में संग्रहीत किया जाता है, और वेट_ओरिग द्वारा गुणा किए गए परिणाम को वजन में संग्रहीत किया जाता है।

**ध्यान दें:** प्रूनिंग ऑपरेशन के बाद वजन अब मॉड्यूल का एक पैरामीटर नहीं है, बल्कि सिर्फ मॉड्यूल की एक विशेषता है।

प्रत्येक प्रूनिंग ऑपरेशन के लिए, मॉडल प्रूनिंग के लिए एक विशिष्ट _forward_pre_hooks फ़ंक्शन के अनुरूप होगा, जो निष्पादित प्रूनिंग ऑपरेशन को संग्रहीत करता है।

In [354]:
# प्रिंट_फॉरवर्ड_प्री_हुक
print(module._forward_pre_hooks)

OrderedDict([(327, <torch.nn.utils.prune.RandomStructured object at 0x00000235D8EFF1C0>)])


#### 1.1.2 सामान्य संरचित छंटाई (एलएन_संरचित)
किसी मॉडल के मापदंडों को कई बार काटा जा सकता है, जिसे पुनरावृत्तीय छंटाई कहा जाता है। उपरोक्त चरणों ने conv1 पर यादृच्छिक संरचित छंटाई की है। इसके बाद, यह देखने के लिए कि क्या होता है, उस पर मानक संरचित छंटाई करें।

In [355]:
# पहला पैरामीटर: मॉड्यूल, कांट-छांट किए जाने वाले विशिष्ट मॉड्यूल का प्रतिनिधित्व करता है, यहां मॉड्यूल=model.conv1 को संदर्भित करता है,
# समझाएं कि प्रूनिंग पहली कन्वोल्यूशनल परत पर की जानी है।
# दूसरा पैरामीटर: नाम, दर्शाता है कि चयनित मॉड्यूल में कौन से पैरामीटर को काटा जाना है।
# यह नाम = "वजन" पर सेट है, जिसका अर्थ है कि नेटवर्क में वजन कम किया गया है, पूर्वाग्रह नहीं।
# तीसरा पैरामीटर: राशि, मॉडल में एक विशिष्ट अनुपात या मापदंडों की पूर्ण संख्या पर की जाने वाली छंटाई का प्रतिनिधित्व करता है।
# राशि 0.0-1.0 के बीच एक फ्लोट मान है, जो अनुपात का प्रतिनिधित्व करता है, या एक सकारात्मक पूर्णांक है, जो ट्रिम किए जाने वाले मापदंडों की निर्दिष्ट संख्या का प्रतिनिधित्व करता है।
# चौथा पैरामीटर: n, मानक प्रकार का प्रतिनिधित्व करता है, जहां n=2 L2 मानक का प्रतिनिधित्व करता है।
# पांचवां पैरामीटर: मंद, कांट-छांट किए जाने वाले चैनल के आयाम सूचकांक का प्रतिनिधित्व करता है।

prune.ln_structured(module, name="weight", amount=0.5, n=2, dim=0)

#मॉडल पैरामीटर दोबारा प्रिंट करें
print(" model state_dict keys:")
print(model.state_dict().keys())
print('*'*50)

print(" module named_parameters:")
print(list(module.named_parameters()))
print('*'*50)

print(" module named_buffers:")
print(list(module.named_buffers()))
print('*'*50)

print(" module weight:")
print(module.weight)
print('*'*50)

print(" module _forward_pre_hooks:")
print(module._forward_pre_hooks)

 model state_dict keys:
odict_keys(['conv1.bias', 'conv1.weight_orig', 'conv1.weight_mask', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])
**************************************************
 module named_parameters:
[('bias', Parameter containing:
tensor([-0.0893, -0.1464, -0.1101, -0.0076,  0.1493, -0.0418],
       requires_grad=True)), ('weight_orig', Parameter containing:
tensor([[[[ 0.0220,  0.1789, -0.0544, -0.0713,  0.0478],
          [ 0.1995, -0.0415,  0.0288, -0.1431,  0.1057],
          [ 0.1600,  0.0248, -0.1903, -0.0242, -0.1961],
          [-0.0211,  0.0257, -0.1116, -0.1678,  0.0611],
          [ 0.0012,  0.0420, -0.1725, -0.1265, -0.1075]]],


        [[[-0.0540, -0.1928, -0.0355, -0.0075, -0.1481],
          [ 0.0135,  0.0192,  0.0082, -0.0120, -0.0164],
          [-0.0435, -0.1488,  0.1092, -0.0041,  0.1960],
          [-0.1045, -0.0136,  0.0398, -0.1286,  0.0617],
          [-0.0091,  0.0466,  0.1827,  0.16

निष्कर्ष: पुनरावृत्त प्रूनिंग कई प्रूनिंग कर्नेल को एक प्रूनिंग कर्नेल में क्रमबद्ध करने के बराबर है। नए मास्क मैट्रिक्स को PruningContainer में कंप्यूट_मास्क विधि का उपयोग करके पुराने मास्क मैट्रिक्स के साथ जोड़ा जाता है। अंत में, केवल एक वेट_ओरिग और वेट_मास्क होता है।

मैं संपूर्ण काट-छाँट इतिहास कैसे देख सकता हूँ? मॉड्यूल._फॉरवर्ड_प्री_हूक्स मॉडल के आगे प्रसार से पहले कस्टम संचालन करने के लिए एक तंत्र है। प्रदर्शन की गई प्रूनिंग विधियां यहां दर्ज की गई हैं।

In [356]:
#प्रिंट प्रूनिंग इतिहास
for hook in module._forward_pre_hooks.values():
    if hook._tensor_name == "weight":  
        break

print(list(hook))  

[<torch.nn.utils.prune.RandomStructured object at 0x00000235D8EFF1C0>, <torch.nn.utils.prune.LnStructured object at 0x00000235D9381F10>]


#### 1.1.3 यादृच्छिक असंरचित छंटाई (यादृच्छिक_असंरचित)
मॉडल के किसी भी उपसंरचना को वजन घटाने के अलावा पूर्वाग्रह को भी काटा जा सकता है।

In [357]:
# पहला पैरामीटर: मॉड्यूल, कांट-छांट किए जाने वाले विशिष्ट मॉड्यूल का प्रतिनिधित्व करता है, यहां मॉड्यूल=model.conv1 को संदर्भित करता है,
# समझाएं कि प्रूनिंग पहली कन्वोल्यूशनल परत पर की जानी है।
# दूसरा पैरामीटर: नाम, दर्शाता है कि चयनित मॉड्यूल में कौन से पैरामीटर को काटा जाना है।
# यह नाम = "वजन" पर सेट है, जिसका अर्थ है कि नेटवर्क में वजन कम किया गया है, पूर्वाग्रह नहीं।
# तीसरा पैरामीटर: राशि, मॉडल में एक विशिष्ट अनुपात या मापदंडों की पूर्ण संख्या पर की जाने वाली छंटाई का प्रतिनिधित्व करता है।
# राशि 0.0-1.0 के बीच एक फ्लोट मान है, जो अनुपात का प्रतिनिधित्व करता है, या एक सकारात्मक पूर्णांक है, जो ट्रिम किए जाने वाले मापदंडों की निर्दिष्ट संख्या का प्रतिनिधित्व करता है।

prune.random_unstructured(module, name="bias", amount=1)

#मॉडल पैरामीटर दोबारा प्रिंट करें
print(" model state_dict keys:")
print(model.state_dict().keys())
print('*'*50)

print(" module named_parameters:")
print(list(module.named_parameters()))
print('*'*50)

print(" module named_buffers:")
print(list(module.named_buffers()))
print('*'*50)

print(" module bias:")
print(module.bias)
print('*'*50)

print(" module _forward_pre_hooks:")
print(module._forward_pre_hooks)

 model state_dict keys:
odict_keys(['conv1.weight_orig', 'conv1.bias_orig', 'conv1.weight_mask', 'conv1.bias_mask', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])
**************************************************
 module named_parameters:
[('weight_orig', Parameter containing:
tensor([[[[ 0.0220,  0.1789, -0.0544, -0.0713,  0.0478],
          [ 0.1995, -0.0415,  0.0288, -0.1431,  0.1057],
          [ 0.1600,  0.0248, -0.1903, -0.0242, -0.1961],
          [-0.0211,  0.0257, -0.1116, -0.1678,  0.0611],
          [ 0.0012,  0.0420, -0.1725, -0.1265, -0.1075]]],


        [[[-0.0540, -0.1928, -0.0355, -0.0075, -0.1481],
          [ 0.0135,  0.0192,  0.0082, -0.0120, -0.0164],
          [-0.0435, -0.1488,  0.1092, -0.0041,  0.1960],
          [-0.1045, -0.0136,  0.0398, -0.1286,  0.0617],
          [-0.0091,  0.0466,  0.1827,  0.1655,  0.0727]]],


        [[[ 0.1216, -0.0833, -0.1491, -0.1143,  0.0113],
          [ 0.0452,  0.1

निष्कर्ष: मॉड्यूल के विभिन्न पैरामीटर सेटों पर अलग-अलग प्रूनिंग रणनीतियों को लागू करते हुए, आप पा सकते हैं कि न केवल वेट_ओरिग, बल्कि बायस_ओरिग भी मॉडल पैरामीटर स्टेट_डिक्ट और नेम_पैरामीटर में शामिल है। नेम_बफ़र्स पैरामीटर में, वेट_मास्क और बायस_मास्क भी एक ही समय में दिखाई देते हैं।
अंत में, क्योंकि हमने दो अलग-अलग प्रकार के मापदंडों पर दो अलग-अलग प्रूनिंग फ़ंक्शन लागू किए थे, दो अलग-अलग फ़ंक्शन परिणाम भी _forward_pre_hooks में मुद्रित किए गए थे।

#### 1.1.4 सामान्य असंरचित छंटाई (एल1_असंरचित)
हमने निर्दिष्ट conv1 परत के वजन और पूर्वाग्रह की छंटाई के विभिन्न तरीके पेश किए हैं। क्या हम एक ही समय में मल्टी-लेयर नेटवर्क के विशिष्ट मापदंडों की छंटाई का समर्थन कर सकते हैं?

In [358]:
#मॉडल के मॉड्यूल मापदंडों को छाँटें
for n, m in model.named_modules():
# मॉडल में सभी संकेंद्रित परतों पर l1_असंरचित प्रूनिंग ऑपरेशन करें, और 20% पैरामीटर प्रूनिंग का चयन करें
    if isinstance(m, torch.nn.Conv2d):
        prune.l1_unstructured(m, name="bias", amount=0.2)
# मॉडल में सभी पूरी तरह से कनेक्टेड परतों पर ln_structured प्रूनिंग ऑपरेशन करें, और 40% पैरामीटर प्रूनिंग का चयन करें
# एलिफ़ इज़इंस्टेंस(मॉड्यूल, टॉर्च.एनएन.रैखिक):
# prune.random_structured(मॉड्यूल, नाम = "वजन", राशि = 0.4, मंद = 0)

#मॉडल पैरामीटर दोबारा प्रिंट करें
print(" model state_dict keys:")
print(model.state_dict().keys())
print('*'*50)

print(" module named_parameters:")
print(list(module.named_parameters()))
print('*'*50)

print(" module named_buffers:")
print(list(module.named_buffers()))
print('*'*50)

print(" module weight:")
print(module.weight)
print('*'*50)

print(" module bias:")
print(module.bias)
print('*'*50)

print(" module _forward_pre_hooks:")
print(module._forward_pre_hooks)

 model state_dict keys:
odict_keys(['conv1.weight_orig', 'conv1.bias_orig', 'conv1.weight_mask', 'conv1.bias_mask', 'conv2.weight', 'conv2.bias_orig', 'conv2.bias_mask', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])
**************************************************
 module named_parameters:
[('weight_orig', Parameter containing:
tensor([[[[ 0.0220,  0.1789, -0.0544, -0.0713,  0.0478],
          [ 0.1995, -0.0415,  0.0288, -0.1431,  0.1057],
          [ 0.1600,  0.0248, -0.1903, -0.0242, -0.1961],
          [-0.0211,  0.0257, -0.1116, -0.1678,  0.0611],
          [ 0.0012,  0.0420, -0.1725, -0.1265, -0.1075]]],


        [[[-0.0540, -0.1928, -0.0355, -0.0075, -0.1481],
          [ 0.0135,  0.0192,  0.0082, -0.0120, -0.0164],
          [-0.0435, -0.1488,  0.1092, -0.0041,  0.1960],
          [-0.1045, -0.0136,  0.0398, -0.1286,  0.0617],
          [-0.0091,  0.0466,  0.1827,  0.1655,  0.0727]]],


        [[[ 0.1216, -0.0833, -0.1491, -0.1143,  0.0113],


इसके बाद, मॉडल का वजन काट दिया जाता है और स्थायी रूप से हटा दिया जाता है। पिछले छंटनी चरण के बाद, मूल वजन 'वेट_ओरिग' बन गया है, और वजन 'वेट_ओरिग' और मास्क मैट्रिक्स को गुणा करने का परिणाम है, और एक विशेषता बन जाता है देखें कि हटाने के बाद क्या परिवर्तन हुए?

In [359]:
# मॉड्यूल के वजन पर छंटाई और स्थायी निष्कासन करें।
for n, m in model.named_modules():
    if isinstance(m, torch.nn.Conv2d):
        prune.remove(m, 'bias')

# कन्व 1 के वजन पर छंटाई और स्थायी निष्कासन करें
prune.remove(module, 'weight')
print('*'*50)

#काटे गए मॉडल का राज्य शब्दकोश प्रिंट करें
print(" model state_dict keys:")
print(model.state_dict().keys())
print('*'*50)

#मॉडल पैरामीटर दोबारा प्रिंट करें
print(" model named_parameters:")
print(list(module.named_parameters()))
print('*'*50)

#मॉडल मास्क बफ़र्स पैरामीटर को फिर से प्रिंट करें
print(" model named_buffers:")
print(list(module.named_buffers()))
print('*'*50)

#मॉडल के _फॉरवर्ड_प्री_हुक को फिर से प्रिंट करें
print(" model forward_pre_hooks:")
print(module._forward_pre_hooks)

**************************************************
 model state_dict keys:
odict_keys(['conv1.bias', 'conv1.weight', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])
**************************************************
 model named_parameters:
[('bias', Parameter containing:
tensor([-0.0893, -0.1464, -0.0000, -0.0000,  0.1493, -0.0418],
       requires_grad=True)), ('weight', Parameter containing:
tensor([[[[ 0.0000,  0.0000, -0.0000, -0.0000,  0.0000],
          [ 0.0000, -0.0000,  0.0000, -0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000, -0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0000, -0.0000, -0.0000]]],


        [[[-0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
          [ 0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000,  0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000,  0.0000, -0.0000,  0.0000],
          [-0.0000,  0.0

निष्कर्ष: मॉडल के वजन और पूर्वाग्रह पर हटाने का ऑपरेशन करने के बाद, मॉडल पैरामीटर सेट में वजन_ओरिग और पूर्वाग्रह_ओरिग गायब हो जाते हैं और वजन और पूर्वाग्रह बन जाते हैं, यह दर्शाता है कि प्रूनिंग ने स्थायी रूप से प्रभाव डाला है, इसे नामित_बफ़र्स टेंसर की छपाई से देखा जा सकता है वह केवल [ ], क्योंकि वज़न और पूर्वाग्रह को छिपाने के लिए उपयोग किए जाने वाले वेट_मास्क और बायस-मास्क प्रभावी हो गए हैं और अब उन्हें बनाए रखने की आवश्यकता नहीं है।
उसी तरह, _forward_pre_hooks में केवल एक खाली शब्दकोश बचा है। वजन और पूर्वाग्रह फिर से पैरामीटर बन जाते हैं, और छंटाई स्थायी हो जाती है।

## 2.ग्लोबल प्रूनिंग (ग्लोबल प्रूनिंग)

स्थानीय प्रूनिंग के चार तरीके पहले ही पेश किए जा चुके हैं, लेकिन काफी हद तक, आपको अपने अनुभव के आधार पर नेटवर्क की एक निश्चित परत को प्रून करने का निर्णय लेना होगा।
एक अधिक सामान्य प्रूनिंग रणनीति ग्लोबल प्रूनिंग का उपयोग करना है, जो पूरे नेटवर्क के परिप्रेक्ष्य से प्रूनिंग करती है। ग्लोबल प्रूनिंग का उपयोग करने के बाद, विभिन्न परतों की प्रूनिंग का प्रतिशत भिन्न हो सकता है।

In [363]:
model = LeNet().to(device=device)

# सबसे पहले आरंभिक मॉडल का राज्य शब्दकोश प्रिंट करें
print(model.state_dict().keys())
print('*'*50)

# एक पैरामीटर सेट बनाएं और तय करें कि कौन सी परतें और कौन से पैरामीटर सेट प्रूनिंग में भाग लेते हैं
parameters_to_prune = (
            (model.conv1, 'weight'),
            (model.conv2, 'weight'),
            (model.fc1, 'weight'),
            (model.fc2, 'weight'))

# प्रूनिंग ऑपरेशन करने के लिए प्रून में ग्लोबल प्रूनिंग फ़ंक्शन ग्लोबल_अनस्ट्रक्चर्ड को कॉल करें
prune.global_unstructured(parameters_to_prune, pruning_method=prune.L1Unstructured, amount=0.2)

#काटे गए मॉडल का राज्य शब्दकोश प्रिंट करें
print(model.state_dict().keys())

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])
**************************************************
odict_keys(['conv1.bias', 'conv1.weight_orig', 'conv1.weight_mask', 'conv2.bias', 'conv2.weight_orig', 'conv2.weight_mask', 'fc1.bias', 'fc1.weight_orig', 'fc1.weight_mask', 'fc2.bias', 'fc2.weight_orig', 'fc2.weight_mask', 'fc3.weight', 'fc3.bias'])


मॉडल को काटने के बाद, अलग-अलग परतों में वजन मापदंडों के अलग-अलग अनुपात होंगे, इसे प्रिंट करने के लिए कोड का उपयोग करें और देखें:

In [364]:
print(
    "Sparsity in conv1.weight: {:.2f}%".format(
    100. * float(torch.sum(model.conv1.weight == 0))
    / float(model.conv1.weight.nelement())
    ))

print(
    "Sparsity in conv2.weight: {:.2f}%".format(
    100. * float(torch.sum(model.conv2.weight == 0))
    / float(model.conv2.weight.nelement())
    ))

print(
    "Sparsity in fc1.weight: {:.2f}%".format(
    100. * float(torch.sum(model.fc1.weight == 0))
    / float(model.fc1.weight.nelement())
    ))

print(
    "Sparsity in fc2.weight: {:.2f}%".format(
    100. * float(torch.sum(model.fc2.weight == 0))
    / float(model.fc2.weight.nelement())
    ))


print(
    "Global sparsity: {:.2f}%".format(
    100. * float(torch.sum(model.conv1.weight == 0)
               + torch.sum(model.conv2.weight == 0)
               + torch.sum(model.fc1.weight == 0)
               + torch.sum(model.fc2.weight == 0))
         / float(model.conv1.weight.nelement()
               + model.conv2.weight.nelement()
               + model.fc1.weight.nelement()
               + model.fc2.weight.nelement())
    ))

Sparsity in conv1.weight: 5.33%
Sparsity in conv2.weight: 17.25%
Sparsity in fc1.weight: 22.03%
Sparsity in fc2.weight: 14.67%
Global sparsity: 20.00%


निष्कर्ष: वैश्विक प्रूनिंग रणनीति का उपयोग करते समय (यह मानते हुए कि पैरामीटर के अनुपात का 20% प्रूनिंग में भाग लेते हैं), मॉडल के समग्र मापदंडों में से केवल 20% को प्रून करने की गारंटी दी जाती है, प्रत्येक परत की विशिष्ट स्थिति विशिष्ट द्वारा निर्धारित की जाती है मॉडल का पैरामीटर वितरण आएं और निर्णय लें।

## 3. उपयोगकर्ता-परिभाषित प्रूनिंग (कस्टम प्रूनिंग)।

प्रूनिंग मॉडल क्लास बेसप्रुनिंगमेथोड () को इनहेरिट करके प्रूनिंग करता है। कई आंतरिक तरीके हैं: कॉल, अप्लाई_मास्क, अप्लाई, प्रून, रिमूव, आदि। कस्टम प्रूनिंग नियम को पूरा करने के लिए दो फ़ंक्शन, __init__ (कन्स्ट्रक्टर) और कंप्यूट_मास्क को लागू किया जाना चाहिए समायोजन।

In [ ]:
# प्रूनिंग विधि को अनुकूलित करने के लिए क्लास को prune.BasePruningMethod इनहेरिट करना होगा
class custom_prune(prune.BasePruningMethod):
# इस तकनीक द्वारा कार्यान्वित छंटाई के प्रकार को निर्दिष्ट करें (समर्थित विकल्प वैश्विक, संरचित और असंरचित हैं)
    PRUNING_TYPE = "unstructured"

# आंतरिक रूप से कंप्यूट_मास्क फ़ंक्शन को लागू करें, पूरी तरह से परिभाषित प्रूनिंग नियम, अनिवार्य रूप से वजन मापदंडों को कैसे छिपाएं
    def compute_mask(self, t, default_mask):
        mask = default_mask.clone()
# यहां परिभाषित नियम हर दूसरे पैरामीटर को छुपाने के लिए है। अंत में, प्रूनिंग में शामिल 50% पैरामीटर को छुपा दिया जाता है।
        mask.view(-1)[::2] = 0
        return mask

# प्रूनिंग विधि के फ़ंक्शन को कस्टमाइज़ करें, और सीधे आंतरिक रूप से प्रूनिंग क्लास के मेथड अप्लाई को कॉल करें
def custome_unstructured_pruning(module, name):
    custom_prune.apply(module, name)
    return module

In [365]:
import time
# मॉडल वर्ग को त्वरित करें
model = LeNet().to(device=device)

start = time.time()
# मॉडल में पहली पूरी तरह से कनेक्टेड लेयर fc1 में बायस बायस पर कस्टम प्रूनिंग करने के लिए कस्टम प्रूनिंग विधि के फ़ंक्शन को कॉल करें
custome_unstructured_pruning(model.fc1, name="bias")

# सफल प्रूनिंग का सबसे बड़ा संकेत बायस_मास्क पैरामीटर का होना है
print(model.fc1.bias_mask)

#कस्टम प्रूनिंग में लगने वाले समय को प्रिंट करें
duration = time.time() - start
print(duration * 1000, 'ms')

tensor([0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.])
2.996683120727539 ms


निष्कर्ष: मुद्रित बायस_मास्क टेंसर प्रत्येक दूसरे बिट को पूर्वनिर्धारित तरीके से मास्क कर देता है, जिसमें 0 और 1 बारी-बारी से दिखाई देते हैं जब रिमूवल ऑपरेशन बाद में किया जाता है, तो मूल बायस_ओरिग में वजन हर बार इसी तरह से प्रून हो जाएगा।